In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# Let's assume you have these
all_characters = "abcdefghijklmnopqrstuvwxyz"
n_characters = len(all_characters)

# Map from char to index and vice-versa
char_to_idx = {ch: idx for idx, ch in enumerate(all_characters)}
idx_to_char = {idx: ch for idx, ch in enumerate(all_characters)}

# Helper to encode a string to tensor of indices
def string_to_tensor(string):
    tensor = torch.zeros(len(string), dtype=torch.long)
    for i, ch in enumerate(string):
        tensor[i] = char_to_idx[ch]
    return tensor

# Helper to decode tensor to string
def tensor_to_string(tensor):
    return ''.join(idx_to_char[idx.item()] for idx in tensor)

# Vanilla RNN Model
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input_char, hidden):
        # input_char: (batch_size=1, input_size)
        # hidden: (batch_size=1, hidden_size)
        combined = torch.cat((input_char, hidden), 1) # concatenate
        hidden = torch.tanh(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

# Instantiate
n_hidden = 128
rnn = VanillaRNN(n_characters, n_hidden, n_characters)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

# Example dataset
training_data = open('indian_names.txt', 'r').read().splitlines()
#["arjun", "anita", "bharat", "devika", "rajat", "suman", "priya"]

# Training loop
def train(input_line):
    rnn.train()
    optimizer.zero_grad()

    input_tensor = string_to_tensor(input_line[:-1])  # all chars except last
    target_tensor = string_to_tensor(input_line[1:])  # predict next char

    hidden = rnn.init_hidden()

    loss = 0
    for i in range(input_tensor.size(0)):
        input_onehot = F.one_hot(input_tensor[i], num_classes=n_characters).float().unsqueeze(0)
        output, hidden = rnn(input_onehot, hidden)
        loss += criterion(output, target_tensor[i].unsqueeze(0))

    loss.backward()
    optimizer.step()

    return loss.item() / input_tensor.size(0)

# Generate new names
def sample(start_letter='a', max_length=10):
    rnn.eval()
    with torch.no_grad():
        input_tensor = string_to_tensor(start_letter)
        input_onehot = F.one_hot(input_tensor[0], num_classes=n_characters).float().unsqueeze(0)
        hidden = rnn.init_hidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(input_onehot, hidden)
            output_dist = F.softmax(output.view(-1), dim=0)
            top_i = torch.multinomial(output_dist, 1)[0]

            predicted_char = idx_to_char[top_i.item()]
            output_name += predicted_char

            input_onehot = F.one_hot(top_i, num_classes=n_characters).float().unsqueeze(0)

        return output_name

# Training
n_epochs = 20000

for epoch in range(1, n_epochs + 1):
    name = random.choice(training_data)
    loss = train(name)

    if epoch % 200 == 0:
        print(f"Epoch {epoch} Loss: {loss:.4f}")
        print("Generated:", sample(random.choice(all_characters)))
        print()



Epoch 200 Loss: 3.4424
Generated: suudhaiauna

Epoch 400 Loss: 2.2950
Generated: qmanjitranh

Epoch 600 Loss: 2.8994
Generated: tyaninaranh

Epoch 800 Loss: 3.2802
Generated: qayolalgush

Epoch 1000 Loss: 2.1609
Generated: cihalolmire

Epoch 1200 Loss: 2.4945
Generated: arktrpiruja

Epoch 1400 Loss: 2.6385
Generated: qpralavarit

Epoch 1600 Loss: 2.7282
Generated: obhoreallee

Epoch 1800 Loss: 2.5610
Generated: vnshadyyara

Epoch 2000 Loss: 2.1994
Generated: furamanaira

Epoch 2200 Loss: 2.3946
Generated: aramdsanijd

Epoch 2400 Loss: 2.3242
Generated: zgyanjarich

Epoch 2600 Loss: 2.8852
Generated: bidnilgaras

Epoch 2800 Loss: 1.8381
Generated: pantundttdu

Epoch 3000 Loss: 2.1101
Generated: lvijhanadin

Epoch 3200 Loss: 2.4436
Generated: zunatychrbi

Epoch 3400 Loss: 2.8381
Generated: layannsheri

Epoch 3600 Loss: 2.6136
Generated: naashalesup

Epoch 3800 Loss: 1.7908
Generated: jidindunwer

Epoch 4000 Loss: 3.2840
Generated: gohajakiwat

Epoch 4200 Loss: 2.2251
Generated: punindatu